In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord, get_sun
from astropy.coordinates import Angle
from astropy import units as u
from alerts import get_alerts, make_alerts
from datetime import datetime, date
from style import output_folder, big_fontsize, base_width, base_height, dpi, CB_color_cycle
import seaborn as sns
import json
from astropy.time import Time

In [2]:
alerts = get_alerts()

In [96]:
ic16mask = (alerts.Event == 'IC160731A') & (alerts.Class == 'HESE')
hese_mask = alerts.Class == 'HESE'
m = alerts.observed & ~ic16mask &~ hese_mask

unc = {k: np.array([tuple(l[f'{k} Unc (rectangle) float']) if not ((l.Event == 'IC190629A') and (k == 'RA'))
                   else [-np.pi, np.pi]
                   for i, l in alerts[m].iterrows()])
        for k in ['RA', 'Dec']
}

perc = alerts[m]['14d coverage']
delta = {k: (iunc.T + np.array(alerts[k][m])).T
        for k, iunc in unc.items()}

dra = abs(delta['RA'][:,1] - delta['RA'][:,0])
dsindec = abs(np.sin(np.radians(delta['Dec'][:,1])) - np.sin(np.radians(delta['Dec'][:,0])))
areas = 180/np.pi * dra * dsindec * perc/100

complete_area_observed = sum(areas)
print(f"Observed {complete_area_observed} sqd in total!")

Observed 1209.7099105888599 sqd in total!


In [106]:
tdes_per_year = 3
tde_livetime = 500
tdes_on_sky = tdes_per_year * tde_livetime/365
sky_area = 41252.96
chance_coincidence = tdes_on_sky / sky_area * complete_area_observed

print(f"{tdes_on_sky} TDEs on the sky at any time.")
print(f"chance coincidence = {chance_coincidence:.2f}")


4.109589041095891 TDEs on the sky at any time.
chance coincidence = 0.12
